# Applied Data Science Capstone - Week 3 assignment

## Importing table data into Pandas as Dataframe

In [7]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

Neighbourhood=pd.DataFrame(df[0])
Neighbourhood

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


## The Number of "Not assigned" entries in the Borough and Neighbourhood columns

In [8]:
Neighbourhood[Neighbourhood['Borough']=="Not assigned"].count()

Postcode         77
Borough          77
Neighbourhood    77
dtype: int64

In [9]:
Neighbourhood[Neighbourhood['Neighbourhood']=="Not assigned"].count()

Postcode         78
Borough          78
Neighbourhood    78
dtype: int64

In [10]:
Neighbourhood[(Neighbourhood['Neighbourhood']=='Not assigned') & (Neighbourhood['Borough'] != 'Not assigned') ]

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


## Exclude "Not assigned" entries from the Borough column

I named another df to carry entries with their "Borough" coloum that are NOT "Not assigned".
Then I found the "Neighbourhood" column of this df doesn't contain "Not assigned" data.

To verify that, I built a little test if...elif code block.

In [11]:
neigh = Neighbourhood[Neighbourhood['Borough']!= "Not assigned"]
print(neigh.count())
neigh.head(10)

Postcode         210
Borough          210
Neighbourhood    210
dtype: int64


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [12]:
neigh[neigh['Neighbourhood']=="Not assigned"].count()

Postcode         1
Borough          1
Neighbourhood    1
dtype: int64

Here is just some codes i was trying to replace the "Not assigned" entries. I suspected I did it incorrectly, so I tried to replace the "Not assigned" entries from the Neighbourhood DataFrame. Then I am sure the replace() function was used correctly.

In [21]:
neigh2 = neigh

neigh2['Neighbourhood'].replace('Not assigned',neigh2['Borough'],inplace = True)        
neigh2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [14]:
neigh2[neigh2['Neighbourhood']=="Not assigned"].count()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

## To join the Neighbourhoods with the same Borough with comma

In [15]:
neigh3 = neigh.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))

neigh3.reset_index(level=['Postcode','Borough'], inplace=True)


neigh3

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


## To Merge the Lat&Lng data with the list

In [19]:
PostCode = pd.read_csv('https://cocl.us/Geospatial_data')
PostCode

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
neigh4 = pd.merge(neigh3,PostCode, left_on = 'Postcode',right_on = 'Postal Code')
neigh4

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437
